In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as data
import time

In [ ]:
N_IMG = 12415

In [ ]:
! tar -zxvf folder

In [ ]:
def evaluate(yt, yp, num_classes=10):
  C=(yt*num_classes+yp).bincount(minlength=num_classes**2).view(num_classes,num_classes).float()
  return {
      'Acc': C.diag().sum().item()/yt.shape[0],
      'mAcc': (C.diag()/C.sum(-1)).mean().item(),
  }

In [ ]:
def train_shallow(model, tr_images, tr_lables, vl_images, vl_lables, method, exec_time, scores):

  #time tracking
  start=time.time()
  model.fit(tr_images, tr_lables)
  end=time.time()
  exec_time['train'][method]= end-start

  yp = model.predict(tr_images)  

  print("Performance on training: {}".format(evaluate(tr_lables, yp)))

  
  scores[method] = evaluate(tr_lables, yp)

  start=time.time()
  yp = model.predict(vl_images)
  end=time.time()

  exec_time['val'][method]= end-start

  print("Performance on validation: {}".format(evaluate(vl_lables, yp)))



In [ ]:
def compare(scores, exec_time):
  models = list(scores.keys())
  plot_idx = 1

  for metric in ['Acc', 'mAcc']:
    for dataset in ['train', 'val']:

      plt.subplot(2,2,plot_idx)

      s = torch.tensor([scores[model][metric] for model in models])
      t = torch.tensor([exec_times[dataset][model] for model in models])

      plt.plot(t.view(1,-1),s.view(1,-1), '*')

      # This parts adds a label to each point in the scatter point, which in turn
      # corresponds to a particular model
      for i in range(len(models)):
        plt.annotate(models[i],(t[i],s[i]))

      # This adds some margin between axis and the points in the plot
      plt.margins(0.3)


      # The following if-the-else removes the y-axis from the second column of 
      # plots
      if dataset=='train':
        plt.ylabel(metric)
      else:
        plt.yticks([])

      # The following instruction enables the xaxis only for the last row of plots
      if metric=="mAcc":
        plt.xlabel('{} time [s]'.format(dataset))
      else:
        plt.xticks([])

      plot_idx+=1


Mean and standard deviation

In [ ]:
mean = 0.0
std = 0.0

train_path = './dataset/train'

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

dataset = torchvision.datasets.ImageFolder(train_path, transform=transform) 
loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

for batch, _ in loader:
  batch = batch.view(batch.size(0), batch.size(1), -1)
  mean += batch.mean(2).sum(0)
  std += batch.std(2).sum(0)

mean /= N_IMG
std /= N_IMG

print(mean)
print(std)

tensor([0.1658, 0.1610, 0.1573])
tensor([0.1218, 0.1106, 0.1036])


Import data

In [ ]:
resized_normal = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(128),
    torchvision.transforms.Normalize(mean=mean, std=std)
    ])

tot_dataset = torchvision.datasets.ImageFolder('/folder', transform=resized_normal)

n_image = 12415
split = int(n_image*20/100)
print('total datset: {}\ntraining set: {}\nvalidation set: {}'.format(n_image, n_image - split, split))
vl_dataset, tr_dataset = data.random_split(tot_dataset, [split, n_image - split])


trDataLoader = torch.utils.data.DataLoader(tr_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
vlDataLoader = torch.utils.data.DataLoader(vl_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)


total datset: 12415
training set: 9932
validation set: 2483


In [ ]:
# train
images = torch.Tensor([])
labels = torch.Tensor([]).int()

for i, l in trDataLoader:
    i = i.reshape(i.size(0), -1)
    images = torch.cat((images, i), 0)
    labels = torch.cat((labels, l), 0)

# valid
imagesVal = torch.Tensor([])
labelsVal = torch.Tensor([]).int()

for i, l in vlDataLoader:
    i = i.reshape(i.size(0), -1)
    imagesVal = torch.cat((imagesVal, i), 0)
    labelsVal = torch.cat((labelsVal, l), 0)

Initialization

In [ ]:
exec_times = {}
exec_times['train'] = {}
exec_times['val'] = {}
scores = {}

Decision Tree

In [ ]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier()
print("Decision Tree")
train_shallow(dtree, images, labels, imagesVal, labelsVal,'DT' , exec_times, scores)

Decision Tree
Performance on training: {'Acc': 0.997564935064935, 'mAcc': 0.9979947209358215}
Performance on validation: {'Acc': 0.36883223684210525, 'mAcc': 0.3294732868671417}


SVM

In [ ]:
from sklearn import svm
svm_model = svm.SVC()

print("Support Vector Machine")
train_shallow(svm_model, images, labels, imagesVal, labelsVal, "SVM", exec_times, scores)

Support Vector Machine


Random Forest

In [ ]:
from sklearn import ensemble
rf_model = ensemble.RandomForestClassifier(n_estimators=20, criterion="entropy")
print("Random Forest")
train_shallow(rf_model, images, labels, imagesVal, labelsVal, "RF", exec_times, scores)

Random Forest
Performance on training: {'Acc': 0.997362012987013, 'mAcc': 0.997242271900177}
Performance on validation: {'Acc': 0.509046052631579, 'mAcc': 0.451945960521698}


In [ ]:
compare(scores, exec_times)